# Imp

> a devilish import module.

In [ ]:
#| default_exp imp

In [ ]:
#| hide
from nbdev.showdoc import *

### Check Modules for Availbility

In [10]:
#| export
import sys, inspect, importlib
from dataclasses import dataclass
from typing import Optional, Dict, Any

In [9]:
importlib.util.find_spec('rich.progress')

ModuleSpec(name='rich.progress', loader=<_frozen_importlib_external.SourceFileLoader object at 0x10be42dd0>, origin='/Users/solst/mambaforge/envs/iza/lib/python3.11/site-packages/rich/progress.py')

https://docs.python.org/3/library/importlib.html#importing-programmatically

In [ ]:
#| export
def programmatic_import(name: str, lazy: bool = False) -> object:
    spec = importlib.util.find_spec(name)
    loader = spec.loader
    if lazy:
        loader = importlib.util.LazyLoader(loader)
    
    module = importlib.util.module_from_spec(spec)
    sys.modules[name] = module
    loader.exec_module(module)
    return module

def is_module_availble(name:str) -> bool:
    if name in sys.modules:
        return True
    elif importlib.util.find_spec(name) is not None:
        return True
    return False

def get_module(name:str, lazy: Optional[bool] = False) -> object:
    if is_module_availble(name):
        return programmatic_import(name, lazy=lazy)    
    raise ModuleNotFoundError

In [ ]:
#| export
@dataclass
class GlobalImport:
    def __enter__(self):
        return self
    
    def __call__(self):        
        self.collector = inspect.getargvalues(
            inspect.getouterframes(
                inspect.currentframe()
            )[1].frame
        ).locals

    def __exit__(self, *args):
        globals().update(self.collector)

In [41]:

import importlib, warnings
from re import sub
import sys
from dataclasses import dataclass
from typing import Any, Dict, Optional

@dataclass
class Importer:
    module: str
    alias: str
    froms: Dict[str, str]
    fallbacks: Dict[str, Any]
    lazy: Optional[bool] = False

    def __post_init__(self):
        self._import_module()

    def _import_module(self):
        try:
            imported = importlib.import_module(self.module)
            sys.modules[self.module] = imported            
            sys.modules[self.alias] = imported

        except ImportError:
            sys.modules[self.module] = self.fallbacks.get(self.module, None)
            sys.modules[self.alias] = self.fallbacks.get(self.alias, None)
            warnings.warn(f'Could not import {self.module} or {self.alias}')
            return

        updates = dict()
        for submodule, imports in self.froms.items():
            for import_item in imports.split(','):
                import_item = import_item.strip()
                given_name, _, alias_name = import_item.partition(' as ')
                
                varname = given_name if not alias_name else alias_name
                
                try:
                    submod_path = ('.'.join([self.module, submodule])).rstrip('.')                    
                    submod = importlib.import_module(submod_path) 
                    varval = getattr(submod, given_name, None)
                    updates[varname] = varval
                              
                    # sys.modules[submod_path] = 
                except ImportError:
                    warnings.warn(f'Could not import {varname} from  {submod_path}')
                    updates[varname] = varval = self.fallbacks.get(varname, None)


        for varname, fallback in self.fallbacks.items():
            if varname not in updates and varname not in sys.modules and varname not in globals():
                updates[varname] = fallback

        globals().update(updates)


imp_rich = Importer(
    'rich',
    alias='rc',
    froms={
        'tree': 'Tree as rich_tree',
        'text': 'Text',
        'markup': 'escape',
        'filesize': 'decimal',
        'console': 'Console',
        'progress': 'Progress',
        '': 'get_console'
    },
    fallbacks={
        'rich_tree': Any,
        'Text': Any,
    },
)


{'rich_tree': <class 'rich.tree.Tree'>, 'Text': <class 'rich.text.Text'>, 'escape': <function escape at 0x107e5c4a0>, 'decimal': <function decimal at 0x10876b420>, 'Console': <class 'rich.console.Console'>, 'Progress': <class 'rich.progress.Progress'>, 'get_console': <function get_console at 0x107b83e20>}


In [23]:
sys.modules['rich.text'].Text

rich.text.Text

In [123]:
#| export
def is_rich_available() -> bool:
    try:
        import rich
        return True
    except ImportError:
        return False

def is_pytorch_available() -> bool: 
    try:
        import torch
        return True
    except ImportError:
        return False

def is_anndata_available() -> bool:    
    try:
        import anndata 
        return True
    except ImportError:
        return False

In [ ]:
Importer(
    'rich',
    froms = dict(
        tree = 'Tree as rich_tree',
        text = 'Text',
        markup = 'escape',
        filesize = 'decimal',
        console = 'get_console, Console',
        progress = 'Progress'        
    ),
    fallbacks = dict(
        rich_tree=Any,
        Text=Any,
        ...
    )
)


In [11]:
Progress

NameError: name 'Progress' is not defined

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()